# Scratch Work

The below code helped me in producing the `util.py` script, which involved data-wrangling in pandas on the climate change dataset.


In [2]:
import pandas as pd


def convert_emission_times(path):
    """Converts all times in the emissions DataFrame into DateTime values."""
    # make a DF of the emissions CSV, wherever it's located
    carbon_df = pd.read_csv(path)
    # convert to strings
    carbon_df['YYYYMM'] = [str(date)[:-2] + "-" + str(date)[-2:] for date in carbon_df['YYYYMM']]
    # remove all dates in the 13th month
    new_df = carbon_df[~carbon_df["YYYYMM"].str.contains('-13')]
    new_df["YYYYMM"] = pd.to_datetime(new_df['YYYYMM'], format="%Y-%m")
    # convert to datetime
    return new_df.fillna(0)

df = convert_emission_times('../Data/carbon-emissions.csv')

/Users/zainraza/Downloads/dev/courses/DS/detecting-fires/env/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [3]:
data = df.loc[
    df['Description'] =='Total Energy Electric Power Sector CO2 Emissions', 
        ['YYYYMM','Value']
    ]

sources = df['Description'].unique()

data = df.loc[df['Description'] == sources, ['YYYYMM','Value']]



ValueError: Lengths must match to compare

In [7]:
df = convert_emission_times('../Data/carbon-emissions.csv')
# pivot the df so each row shows the emissions of each source in that month
new_df = df.pivot(index='YYYYMM', columns='Description', values='Value')
# fill in NaN values for the energy generation with 0 for now
# new_df = new_df.fillna(0)
# type(new_df['Geothermal Energy Electric Power Sector CO2 Emissions'][0])
new_df['Geothermal Energy Electric Power Sector CO2 Emissions'].fillna(0)
values = list()
for i, v in new_df['Geothermal Energy Electric Power Sector CO2 Emissions'].iteritems():
    if v == 'Not Available':
        values.append(0)
    else:
        values.append(v)
new_df['Geothermal Energy Electric Power Sector CO2 Emissions'] = values
new_df.index


/Users/zainraza/Downloads/dev/courses/DS/detecting-fires/env/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


DatetimeIndex(['1973-01-01', '1973-02-01', '1973-03-01', '1973-04-01',
               '1973-05-01', '1973-06-01', '1973-07-01', '1973-08-01',
               '1973-09-01', '1973-10-01',
               ...
               '2015-10-01', '2015-11-01', '2015-12-01', '2016-01-01',
               '2016-02-01', '2016-03-01', '2016-04-01', '2016-05-01',
               '2016-06-01', '2016-07-01'],
              dtype='datetime64[ns]', name='YYYYMM', length=523, freq=None)

In [8]:
import datetime 

selected_date_range = new_df[
    (new_df.index >= datetime.datetime(1973, 1, 1)) &
    (new_df.index <= datetime.datetime(1975, 12, 31))
]

selected_date_range
cols = new_df.columns
cols
requested_trend_data = selected_date_range[cols]

requested_trend_data 

# set index to 0-indexed list
dates = new_df.index
new_df.index = range(len(dates))
new_df['YYYYMM'] = dates
new_df



Description,Coal Electric Power Sector CO2 Emissions,"Distillate Fuel, Including Kerosene-Type Jet Fuel, Oil Electric Power Sector CO2 Emissions",Geothermal Energy Electric Power Sector CO2 Emissions,Natural Gas Electric Power Sector CO2 Emissions,Non-Biomass Waste Electric Power Sector CO2 Emissions,Petroleum Coke Electric Power Sector CO2 Emissions,Petroleum Electric Power Sector CO2 Emissions,Residual Fuel Oil Electric Power Sector CO2 Emissions,Total Energy Electric Power Sector CO2 Emissions,YYYYMM
0,72.076,2.375,0,12.175,Not Available,.128,27.369,24.867,111.621,1973-01-01
1,64.442,2.061,0,11.708,Not Available,.106,23.034,20.867,99.185,1973-02-01
2,64.084,1.171,0,13.994,Not Available,.083,21.034,19.78,99.112,1973-03-01
3,60.842,1.022,0,14.627,Not Available,.13,17.714,16.562,93.183,1973-04-01
4,61.798,.949,0,17.344,Not Available,.167,18.87,17.754,98.012,1973-05-01
...,...,...,...,...,...,...,...,...,...,...
518,72.84,.279,.035,40.525,.943,1.041,1.654,.334,115.997,2016-03-01
519,71.41,.259,.034,39.763,.912,1.096,1.695,.34,113.815,2016-04-01
520,82.51,.34,.035,44.21,.943,1.035,1.743,.368,129.44,2016-05-01
521,115.772,.293,.034,53.567,.912,1.075,1.789,.42,172.074,2016-06-01


In [9]:
def clean_emission_data(path):
    def fill_not_available(dataframe, column_name):
        '''Fills in 0 for all values in the column that are populated by "Not Available."'''
        new_values = list()
        for item in dataframe[column_name]:
        # get the value to append
            value = item
            # if value is NaN, change to 0
            if value == "Not Available":
                value = 0
            # add the value
            new_values.append(value)
        # reassign the column in the df
        dataframe[column_name] = new_values
        # return dataframe
    """Converts all times in the emissions DataFrame into DateTime values."""
    # make a DF of the emissions CSV, wherever it's located
    carbon_df = pd.read_csv(path)
    # store a list of the different sources
    carbon_categories = carbon_df['Description'].unique()
    # convert dates to strings
    carbon_df['YYYYMM'] = [str(date)[:-2] + "-" + str(date)[-2:] for date in carbon_df['YYYYMM']]
    # remove all dates in the 13th month
    new_df = carbon_df[~carbon_df["YYYYMM"].str.contains('-13')]
    # convert to datetime
    new_df["YYYYMM"] = pd.to_datetime(new_df['YYYYMM'], format="%Y-%m")
    # give each energy source it's own column - now the date is the index
    new_df = new_df.pivot(index='YYYYMM', columns='Description', values='Value')
    # remove NaN values
    for category in carbon_categories:
        new_df = fill_not_available(new_df, category)
    return new_df, carbon_categories

emissions_df, carbon_categories  = clean_emission_data('../Data/carbon-emissions.csv')

emissions_df


/Users/zainraza/Downloads/dev/courses/DS/detecting-fires/env/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


TypeError: 'NoneType' object is not subscriptable

In [10]:
dates = emissions_df.index

emissions_df.index = list(range(len(dates)))
emissions_df

emissions_df['YYYYMM'] = dates

emissions_df

NameError: name 'emissions_df' is not defined